In [31]:
import numpy as np
import pandas as pd
import logging

import time

Auseinandernahme der run-Fkt, aber damit lauffähig möglichst mit getauschten solvern

In [32]:
# Parameter aus dem Aufrug von run
depth = 8
time_limit = 1800
criterion = "gini"

# 1 Preprocessing

In [33]:
# Load your training and test datasets
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

print(train_data)

# get features
feature_columns = train_data.columns[1:]

print(feature_columns)

     y  1  2  3  4  5  6  7  8  9  ...  121  122  123  124  125  126  127  \
0    1  0  0  0  0  0  1  0  0  0  ...    1    0    0    0    0    0    0   
1    1  0  0  0  0  0  0  0  0  0  ...    0    1    0    0    0    0    0   
2    1  0  0  0  0  0  1  0  0  0  ...    0    0    0    0    0    0    0   
3    1  0  0  0  0  0  0  0  0  0  ...    0    1    0    0    0    0    0   
4    1  0  0  0  0  0  0  0  0  0  ...    0    1    0    0    0    0    0   
..  .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
155  3  0  0  0  0  1  0  0  0  0  ...    0    0    0    1    0    0    0   
156  3  0  0  0  0  0  0  0  1  0  ...    0    0    0    0    0    0    0   
157  3  0  0  0  0  0  0  1  0  0  ...    0    0    0    0    0    0    0   
158  3  0  0  0  0  0  1  0  0  0  ...    0    0    0    0    0    0    0   
159  3  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    1    0   

     128  129  130  
0      0    0    0  
1      0    0    0  
2      1    

In [34]:
def preprocess_dataframes(train_df: pd.DataFrame, test_df: pd.DataFrame, target_label: str, features: list):
    """
    Rearranges the DataFrames such that the target label becomes the first column,
    and feature names are converted into ordinal numbers.

    Args:
    - train_df: pandas DataFrame containing the training data
    - test_df: pandas DataFrame containing the test data
    - target_label: string representing the target label
    - features: list of strings representing feature names

    Returns:
    - pd.DataFrame: preprocessed training DataFrame
    - pd.DataFrame: preprocessed test DataFrame
    """

    # Move target label to the first column for both train and test DataFrames
    if target_label in train_df.columns:
        train_target_idx = train_df.columns.get_loc(target_label) # find the integer position (index) of the column named target_label within the DataFrame train_df
        train_df_columns = list(train_df.columns)
        train_df_columns = [train_df_columns[train_target_idx]] + train_df_columns[:train_target_idx] + train_df_columns[train_target_idx + 1:] # new arrangement
        train_df = train_df[train_df_columns] # returns a new DataFrame containing only those columns, arranged in the order specified by the list, panmda reordering

    if target_label in test_df.columns:
        test_target_idx = test_df.columns.get_loc(target_label)
        test_df_columns = list(test_df.columns)
        test_df_columns = [test_df_columns[test_target_idx]] + test_df_columns[:test_target_idx] + test_df_columns[test_target_idx + 1:]
        test_df = test_df[test_df_columns]

    # Rename features to ordinal numbers for both train and test DataFrames
    train_df.rename(columns={feature: str(i) for i, feature in enumerate(features, start=1)}, inplace=True) #  DataFrame columns renaming ['y', 'age', 'income', 'score', ...] to ['y', '1', '2', '3', ...]
    test_df.rename(columns={feature: str(i) for i, feature in enumerate(features, start=1)}, inplace=True)

    return train_df, test_df

In [35]:
train, test = preprocess_dataframes(train_df = train_data, test_df = test_data, target_label = "y", features=feature_columns) #aendert in diesem Fall gar nichts

#print(train)

In [36]:
df = pd.concat([train, test]) #pd.concat concatenates (stacks) the DataFrames vertically (by default), meaning it combines them one after the other, increasing the number of rows. By default detects and preserves the column headers (column names)
#print(df)

K = sorted(list(set(df.y))) #K is a sorted list of all unique class labels (target categories) present in the combined training and test data


P = [int(i) for i in list(train.loc[:, train.columns != 'y'].columns)] # all columns != y; .columns retrieves their names, converts them to list and converts them to integers (assumes that string numbers are column names => preprocessing)

print(P)

train.columns = ["y", *P] #renames first coloumn to "y" and unpacks the list P as the names for the remaining columns

# also zusammen mit dem preprocessing, wird train erst zu ['y', '1', '2', '3', ...] und jetzt zu ['y', 1, 2, 3, ...]; also alles außer target label wird int und erste coloumn (die mit target label) wird 'y'


test.columns = ["y", *P]

#result_dict = {}
#main_model_time = time.time()



[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130]


# 2 Generating models

wie folgendes funktioniert

```python
# generate model
    main_model = generate_model(P=P, K=K, data=train, y_idx=0, big_m=big_m, criterion=criterion)
```

wir wollen haben:
```python
main_model = generate_model(P=P, K=K, data=train, y_idx=0, big_m=big_m, criterion=criterion)
```

in generate model werden folgende weitere Funktionen genutzt:

```python
from rollo_oct.oct.tree import generate_nodes, calculate_gini, calculate_misclassification
```

außerdem: Funktionen aus gurobipy

## 2.1 Die tree Funktionen

In [37]:
def generate_nodes(depth: int) -> list:
    """

    :param depth:
    :return:
    """
    nodes = list(range(1, int(round(2 ** (depth + 1))))) #list of all nodes by definition
    print(nodes)
    parent_nodes = nodes[0: 2 ** (depth + 1) - 2 ** depth - 1]
    print(parent_nodes)
    leaf_nodes = nodes[-2 ** depth:]
    print(leaf_nodes)
    return parent_nodes, leaf_nodes

p_nodes, l_nodes = generate_nodes(2) # (for depth 2) returns [1,2,3] and [4,5,6,7]



[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3]
[4, 5, 6, 7]


## 2.2 loss functions

### 2.2.1 Gini

Let
- $D_{jk}^{(r,s)}$: the subset of datapoints
- $n_{jk}^{(r,s)}$: the number of datapoints

for which the leaf decision rule $\{x_j = r \text{ AND } x_k = s\}$ holds

For the same subset $D_{jk}^{(r,s)}$, the Gini impurity can be computed as follows:

$$c_{jk}^{(r,s)} = \frac{n_{jk}^{(r,s)}}{n} \left( 1 - \sum_{c \in C} p_c^2\right)$$

where equation 5:
$$p_c = \frac{\sum_{i \in D_{jk}^{(r,s)}} \mathbb{1}_{\{y_i = c\}}}{n_{jk}^{(r,s)}}$$
is the fraction of datapoints with class label c in the subset.#


In [38]:
import copy

In [39]:
def gini_index(arr: np.array,
               instance_size: int,
               K: list,
               y_idx: int = 0,
               weighted: bool = True):
    """

    :param arr:
    :param instance_size:
    :param K:
    :param y_idx:
    :param weighted:
    :return:
    """
    sum_ = 0
    for k in K:
        sum_ += np.power(len(arr[np.where(arr[:, y_idx] == k)]) / len(arr), 2)
    sum_ = 1 - sum_
    if weighted:
        sum_ = (len(arr) / instance_size) * sum_
    return sum_

Ohne Multiprocess:

In [40]:
def calculate_gini(data: pd.DataFrame,
                   P: list,
                   K: list,
                   nodes: dict) -> dict:
    """

    :param data: The input dataset as a pandas DataFrame.
    :param P: List of feature indices to consider.
    :param K: List of class labels (target categories).
    :param nodes: dictionary containing information about leaf nodes and their paths.
    :return:
    """
    df_arr = np.array(data)
    #print(df_arr)
    n = len(data) #total number of rows
    #print(n)
    gini_dict = dict()
    for leaf_ in nodes["leaf_nodes"]:
        temp = dict() # temporarily stores Gini indices for feature pairs for the current leaf node
        first_var = nodes["leaf_nodes_path"][leaf_][0] #Retrieves the first and second variable for the path leading to the current leaf node
        second_var = nodes["leaf_nodes_path"][leaf_][1]
        for feature_i in P:
            arr = df_arr[np.where((df_arr[:, feature_i] == first_var))] # For each feature (by index in P), filters the data where the feature equals first_var; arr now contains only rows where feature_i equals first_var

            for feature_j in P:
                arr_2 = arr[np.where(arr[:, feature_j] == second_var)] # For each feature,further filters arr to rows where feature_j equals second_var; arr_2 now contains rows where both feature_i equals first_var and feature_j equals second_var
                if len(arr_2) > 0:
                    temp[feature_i, feature_j] = gini_index(arr=arr_2, # If there are rows left after filtering, calculates the Gini index for that subset, Stores the result in temp under the key (feature_i, feature_j)
                                                            instance_size=n,
                                                            K=K,
                                                            weighted=True)
        gini_dict[leaf_] = copy.deepcopy(temp) # copies the temporary dictionary into gini_dict under the current leaf node key
        del temp
        del arr

    return gini_dict

In [ ]:
def gini_index_fast(y):
    """Vectorized Gini calculation for a 1D array of labels."""
    _, counts = np.unique(y, return_counts=True) #returns the unique labels in y and how many times each appears, counts is an array of the number of occurrences for each unique label
    probs = counts / len(y)
    return 1 - np.sum(probs ** 2)
    #probs = counts / counts.sum()
    #if counts.sum() == len(y):
        #return 1 - np.sum(probs ** 2)

def calculate_gini_fast(data: pd.DataFrame, P: list, K: list, nodes: dict) -> dict:
    """
    :param data: pd.DataFrame — The dataset
    :param P: list of Indices of features to consider
    :param K: list Unique class labels
    :param nodes: dict — Contains: "leaf_nodes": list of leaf node identifiers. leaf_nodes_path": dict mapping each leaf node to a path (list of values for features).
    :return:
    """
    gini_dict = dict()
    n = len(data) #Total number of rows in the dataset
    for leaf_ in nodes["leaf_nodes"]:
        temp = dict()
        #These are the feature values that define the path to this leaf (e.g., in a decision tree, the values that must be matched to reach this leaf):
        first_var = nodes["leaf_nodes_path"][leaf_][0] #{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
        second_var = nodes["leaf_nodes_path"][leaf_][1] #first_var, second_var — the first two values in the path to this leaf (used as feature values for filtering)
        for feature_i in P:
            arr = data[data.iloc[:, feature_i] == first_var] # filters to rows where feature_i equals first_var
            for feature_j in P:
                arr_2 = arr[arr.iloc[:, feature_j] == second_var] #Further filters arr to rows where feature_j equals second_var
                if not arr_2.empty:
                    gini = gini_index_fast(arr_2.iloc[:, 0].values) #Calculates the Gini impurity of the labels in the first column (assumed to be the label column).
                    weighted_gini = (len(arr_2) / n) * gini
                    temp[(feature_i, feature_j)] = weighted_gini
        gini_dict[leaf_] = temp
    return gini_dict

In [42]:
leaf_nodes_path = {4: [1, 1], #dict for all leafes in depth 2 tree and der respective feature selection
                       5: [1, 0],
                       6: [0, 1],
                       7: [0, 0]}

depth_gini = 2
parent_nodes, leaf_nodes = generate_nodes(depth_gini) # (for depth 2) returns [1,2,3] and [4,5,6,7]

nodes = dict()
nodes["leaf_nodes"] = leaf_nodes
nodes["leaf_nodes_path"] = leaf_nodes_path

print(nodes)

gini_dict = calculate_gini(data=train, P=P, K=K, nodes=nodes)
calculate_gini(data=train, P=P, K=K, nodes=nodes)

[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3]
[4, 5, 6, 7]
{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}


{4: {(1, 1): np.float64(0.0),
  (1, 11): np.float64(0.0),
  (1, 12): np.float64(0.0),
  (1, 14): np.float64(0.0),
  (1, 16): np.float64(0.0),
  (1, 18): np.float64(0.0),
  (1, 19): np.float64(0.0),
  (1, 20): np.float64(0.0),
  (1, 21): np.float64(0.0),
  (1, 23): np.float64(0.0),
  (1, 24): np.float64(0.0),
  (1, 26): np.float64(0.0),
  (1, 27): np.float64(0.0),
  (1, 28): np.float64(0.0),
  (1, 29): np.float64(0.0),
  (1, 30): np.float64(0.0),
  (1, 31): np.float64(0.0),
  (1, 33): np.float64(0.0),
  (1, 35): np.float64(0.0),
  (1, 36): np.float64(0.0),
  (1, 37): np.float64(0.0),
  (1, 38): np.float64(0.0),
  (1, 40): np.float64(0.0),
  (1, 41): np.float64(0.0),
  (1, 42): np.float64(0.0),
  (1, 43): np.float64(0.0),
  (1, 44): np.float64(0.0),
  (1, 45): np.float64(0.0),
  (1, 46): np.float64(0.0),
  (1, 48): np.float64(0.0),
  (1, 49): np.float64(0.0),
  (1, 52): np.float64(0.0),
  (1, 53): np.float64(0.0),
  (1, 54): np.float64(0.0),
  (1, 55): np.float64(0.0),
  (1, 56): np.floa

In [43]:
#print(gini_dict)
gini_dict[4].get((2, 2),99)

np.float64(0.021875000000000002)

In [44]:
gini_dict = calculate_gini_fast(data=train, P=P, K=K, nodes=nodes)
calculate_gini_fast(data=train, P=P, K=K, nodes=nodes)

{4: {(1, 1): np.float64(0.0),
  (1, 11): np.float64(0.0),
  (1, 12): np.float64(0.0),
  (1, 14): np.float64(0.0),
  (1, 16): np.float64(0.0),
  (1, 18): np.float64(0.0),
  (1, 19): np.float64(0.0),
  (1, 20): np.float64(0.0),
  (1, 21): np.float64(0.0),
  (1, 23): np.float64(0.0),
  (1, 24): np.float64(0.0),
  (1, 26): np.float64(0.0),
  (1, 27): np.float64(0.0),
  (1, 28): np.float64(0.0),
  (1, 29): np.float64(0.0),
  (1, 30): np.float64(0.0),
  (1, 31): np.float64(0.0),
  (1, 33): np.float64(0.0),
  (1, 35): np.float64(0.0),
  (1, 36): np.float64(0.0),
  (1, 37): np.float64(0.0),
  (1, 38): np.float64(0.0),
  (1, 40): np.float64(0.0),
  (1, 41): np.float64(0.0),
  (1, 42): np.float64(0.0),
  (1, 43): np.float64(0.0),
  (1, 44): np.float64(0.0),
  (1, 45): np.float64(0.0),
  (1, 46): np.float64(0.0),
  (1, 48): np.float64(0.0),
  (1, 49): np.float64(0.0),
  (1, 52): np.float64(0.0),
  (1, 53): np.float64(0.0),
  (1, 54): np.float64(0.0),
  (1, 55): np.float64(0.0),
  (1, 56): np.floa

In [ ]:
gini_dict[4].get((2, 2),99)

### 2.2.2 misclassification
xxx

## 2.3 Generierung der Models mit pulp
generate_model mit pulp

hier wird lookahead depth für den solver auf depth 2 hardcoded

In [45]:
"""
from pulp import *

def generate_model_pulp(
        P: list,
        K: list,
        data: pd.DataFrame,
        y_idx: int = 0,
        time_limit: float = 1800, # moved to train_model to give parameter to solver directly
        gap_limit: float = None,
        log_to_console: bool = False,
        big_m: int = 99,
        criterion: str = "gini",
):
    """

    :param criterion:
    :param big_m:
    :param depth:
    :param P:
    :param K:
    :param data:
    :param leaf_nodes_path:
    :param y_idx:
    :param time_limit:
    :param gap_limit:
    :param log_to_console:
    :return:
    """

    # Create parent & leaf nodes

    leaf_nodes_path = {4: [1, 1], #dict for all leafes in depth 2 tree and der respective split conditions
                       5: [1, 0],
                       6: [0, 1],
                       7: [0, 0]}
    depth = 2
    parent_nodes, leaf_nodes = generate_nodes(depth) # (for depth 2) returns [1,2,3] and [4,5,6,7]

    nodes = dict()
    nodes["leaf_nodes"] = leaf_nodes
    nodes["leaf_nodes_path"] = leaf_nodes_path

    print(nodes)

    if criterion == "gini":
        logging.info("Calculating gini..")
        coef_dict = calculate_gini(data=data,
                                   P=P,
                                   K=K,
                                   nodes=nodes)
        
    elif criterion == "misclassification":
        logging.info("Calculating misclassification..")
        coef_dict = calculate_misclassification(data=data,
                                                P=P,
                                                nodes=nodes)
    # init model
    model = LpProblem("RollOCT", LpMinimize) # Sets the objective to be minimized

    # # x[i,j] and y[i,k] as binary variables
    x = dict()
    for i in P:
        for j in P:
            x[i, j] = LpVariable(f'x[{i},{j}]', cat='Binary') #creates variable
            # In PuLP, variables are automatically added to the model when you include them in constraints or the objective function—you do not need to explicitly register them with the model
            # cat='Binary': Specifies the variable type as binary
            # name=f'x[{i}, {j}]': Assigns a name to the variable for easier identification and debugging
            # x[i, j]: Binary variable indicating whether feature i is used for the first split and feature j for the second split

    y = dict()
    for i in P:
        for k in P:
            y[i, k] = LpVariable(f'y[{i},{k}]', cat='Binary')

#lpSum is to gulps what quicksum is to gurobi

    # Constraint 1: Exactly one (i,j) pair is selected
    model += lpSum(x[i,j] for i in P for j in P) == 1, "C1b" # Ensures that exactly one combination of features (i, j) is selected for the first two splits (criteria (1b))
    # Constraint 2: Exactly one (i,k) pair is selected
    model += lpSum(y[i,k] for i in P for k in P) == 1, "C1c"  # implements criteria (1c) same way as above
    # Constraint 3
    for i in P:
        model += lpSum(x[i,j] for j in P) == lpSum(y[i,k] for k in P), f"C1d_{i}" # Links the x and y variables, ensuring that for each feature i, the sum of x[i, j] across j equals the sum of y[i, k] across k. This ensures consistency between the splits (criteria (1d))

    # Constraint 4
    # add big m; Acts as a penalty for invalid splits (if a combination is not present in coef_dict, it uses big_m as a large penalty)

# implements criteria (1a)
# variable x[i, j], y[i, k] is binary, so 1 or 0. So the multiplication makes sense
# .get(key, default): Dictionary method to safely retrieve values, using big_m as a fallback
# coef_dict[4].get((i, j))
# in coef_dict[4].get((i, j) is value of loss function of leaf 4 with features i,j from P. Value is found in dict if it matches the respective split condition of leaf 4 [1, 1]. If not penalty big m is used
# => damit der Wert fuer loss function bei feature combination i,j im coef_dict steht, müssen i,j (nach definition wie coef_dict erstellt wird) dort zur split condition (vorhanden/nicht vorhanden) gematched haben  
# => also man kann in coef_dict[4].get((i, j, big_m) nur Werte für die features finden, die im Datensatz der Kombination [1,1] entsprochen haben. Falls der Wert nicht vorhanden ist (alle i,j Kombinationen werden iteriert), wird stattdessen Wert big_m genommen

    obj = lpSum(
        (coef_dict[4].get((i, j), big_m) + coef_dict[5].get((i, j), big_m)) *
        x[i, j]
        for i in P
        for j in P) + \
          lpSum((coef_dict[6].get((i, k), big_m) + coef_dict[7].get((i, k), big_m)) *
                   y[i, k]
                   for i in P
                   for k in P)

    model += obj, "Objective"


    # #set a time limit (if using a compatible solver like CBC or Gurobi):
    # if time_limit:
    #     m.setParam("TimeLimit", time_limit)
    #     logging.info(f'Setting Time Limit as {time_limit}')

    # if gap_limit is not None:
    #     m.setParam("MipGap", gap_limit) #MipGap: Sets the optimality gap tolerance for early termination
    #     logging.info(f'Setting Optimality Gap as {gap_limit}')

    # m.setParam("LogToConsole", int(log_to_console))
    # logging.info(f'Setting LogToConsole as {log_to_console}')
    # m.update() #Updates the model with all changes.

    model_dict = {
        'model': model,
        'params': {
            'var_x': x,
            'var_y': y,
            'y_idx': y_idx
        },
        'nodes': {
            'leaf_nodes': leaf_nodes,
            'parent_nodes': parent_nodes,
            "leaf_nodes_path": leaf_nodes_path,
        },
        'depth': depth,
        "P": P,
        "K": K
    }
    logging.info('Model generation is done.')

    #model.solve()

    return model_dict

IndentationError: unexpected indent (3504598243.py, line 17)

In [ ]:
main_model_pulp = generate_model_pulp(P=P, K=K, data=train, y_idx=0, big_m=99, criterion="gini")

m = main_model_pulp['model']

print(m)

[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3]
[4, 5, 6, 7]
{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
RollOCT:
MINIMIZE
99.0*x_1,100_ + 99.0*x_1,101_ + 99.0*x_1,102_ + 99.0*x_1,104_ + 99.0*x_1,107_ + 99.0*x_1,10_ + 99.0*x_1,111_ + 99.0*x_1,112_ + 99.0*x_1,121_ + 99.0*x_1,122_ + 99.0*x_1,129_ + 99.0*x_1,130_ + 99.0*x_1,13_ + 99.0*x_1,15_ + 99.0*x_1,17_ + 99.0*x_1,1_ + 99.0*x_1,22_ + 99.0*x_1,25_ + 99.0*x_1,2_ + 99.0*x_1,32_ + 99.0*x_1,34_ + 99.0*x_1,39_ + 99.0*x_1,3_ + 99.0*x_1,47_ + 99.0*x_1,4_ + 99.0*x_1,50_ + 99.0*x_1,51_ + 99.0*x_1,57_ + 99.0*x_1,5_ + 99.0*x_1,61_ + 99.0*x_1,62_ + 99.0*x_1,6_ + 99.0*x_1,70_ + 99.0*x_1,73_ + 99.0*x_1,7_ + 99.0*x_1,81_ + 99.0*x_1,88_ + 99.0*x_1,8_ + 99.0*x_1,95_ + 99.0*x_1,96_ + 99.0*x_1,97_ + 99.0*x_1,98_ + 99.0*x_1,99_ + 99.0*x_1,9_ + 0.008333333333333333*x_10,100_ + 99.02142857142857*x_10,102_ + 99.02142857142857*x_10,103_ + 0.020454545454545447*x_10,104_ + 0.021153846153846155*x_10,105_ + 0.020833333333333325*x

In [ ]:
#params = main_model_pulp['params']

#print(params)
#print(params['var_x'])

In [ ]:
#m.solve(solver=PULP_CBC_CMD(msg=True, timeLimit=time_limit))

#m.solve(solver=PULP_CBC_CMD(msg=True))

#LpStatus[m.status]


In [ ]:
#params = main_model_pulp['params']

#print(params)
#print(params['var_x'])

In [ ]:
#for v in m.variables():
    #print(v.name, "=", v.varValue)

# 3 Trainieren des Models

## 3.1 Pulp Version - Trainieren des Models

hier ist noch time limit broken; der solver muss time limit unterstützen (maybe fixed)

In [ ]:
# main_model = train_model(model_dict=main_model, data=train, P=P)


import time

def train_model_pulp(data: pd.DataFrame,
                model_dict: dict,
                P: list,
                time_limit: float = 1800):
                
    
    """
    :param data:
    :param model_dict: A dictionary containing model components (nodes, parameters, and the Gurobi model)
    :param P: List of feature indices to consider
    :return:
    """

    #initialization
    logging.info("Training..")
    nodes = model_dict['nodes']
    params = model_dict['params'] #is reference to the variables in the model; references them directly. They cary solution after m.optimize()
    m = model_dict['model']


    start_time = time.time()
    m.solve(solver=PULP_CBC_CMD(msg=True, timeLimit=time_limit)) # solves our optimization model
    runtime = time.time() - start_time

    #Check Optimization Status

    if LpStatus[m.status] == "Optimal": 
        logging.info(f'Optimal objective: {m.objective.value()}') # OPTIMAL: Solution found.
        status = m.status
    elif LpStatus[m.status] == "Infeasible": # INFEASIBLE: No solution exists
        logging.info("Model is infeasible")
        return
    elif LpStatus[m.status] == "Unbounded": # UNBOUNDED: The objective can be improved infinitely
        logging.info("Model is unbounded")
        return
    #elif m.status == 9 and m.objVal != -math.inf: #Status 9: Time limit reached (if objective is not -∞)
        #logging.info('Time Limit termination')
        #return
    else:
        logging.info(f'Optimization status {LpStatus[m.status]}')
        return

    logging.info(f"Objective Value: {m.objective.value()}")
    
    
    # Arrange/EXTRACT decision variables WITH SOLUTIONS => dict of solutions (optimization for splits)
    #x and y: Dictionaries mapping pairs (i, j) (key) to the value of the decision variables var_x and var_y (data) from the solution.
    # variables are defined as binary, so they are 0 or 1
    x = {
        (i, j): params['var_x'][i, j].value() #Accesses the attribute of the variable params['var_x'][i, j]; In Gurobi, .X gives the solution value of the variable after optimization
        for i in P for j in P
    }
    y = {
        (i, j): params['var_y'][i, j].value()
        for i in P for j in P
    }



    # Identify Selected Features
    # Sets 'first_level', 'left_second_level', and 'right_second_level' variables => wir bekommen also die Identifikation für die features in jedem Split
    # wir wollen hier über alle variablen die Variablen finden, die mit 1 belegt sind, also laut solver gewählt werden sollen. Aus diesen entnehmen wir die features
    for i in P:
        for j in P:
            if x[i, j] > 0: # Finds which features are selected (where x (or y) is positive (1) ).
                first_level = i
                left_second_level = j
                logging.info(
                    f"First Level Feature: {i} & Second Level Left Feature: "
                    f"{j}, {x[i, j]}={x[i, j]}")
            if y[i, j] > 0:
                right_second_level = j
                logging.info(
                    f"First Level Feature: {i} & Second Level Right Feature:"
                    f" {j}, {y[i, j]}={y[i, j]}")
                
# Determine Leaf Node Predictions, get target classes pf the leafs
# hier wird wieder aus den features ausgedünnt 
    logging.info("Extracting solution..")
    df_arr = np.array(data)
    target_class = dict()
    for leaf_ in nodes["leaf_nodes"]: # iterate through leaf nodes
        first_var = nodes["leaf_nodes_path"][leaf_][0]
        second_var = nodes["leaf_nodes_path"][leaf_][1]
        arr = df_arr[np.where((df_arr[:, first_level] == first_var))] # Selects all rows (observations), but only the column at index first_level. Compares each value in the first_level column to first_var; returns bool: True means the value matches first_var
        # wir wissen nach welchem feature wir root splitten wollen (first_level). Also nehmen wir nur diese Spalte. Dort nehmen wir alle Zeilen, deren Wert first_var entspricht (also Wert aus {0,1})
        if leaf_ in [4, 5]:  # left
            arr_2 = arr[np.where(arr[:, left_second_level] == second_var)] #entsprechend weiter filtern
        elif leaf_ in [6, 7]:  # right
            arr_2 = arr[np.where(arr[:, right_second_level] == second_var)] #entsprechend weiter filtern
        else:
            pass
        values, counts = np.unique(arr_2[:, params["y_idx"]], 
                                   return_counts=True)
        # params["y_idx"] meist= params[0] is coloumn with the target variables (the class label we want to predict))
        # values: An array of the unique class labels present in the filtered data => also alle class labels die nach dem filtern noch übrig sind
        # counts: An array of how many times each unique class label appears in the filtered data.

        if len(counts) > 0:
            target_class[leaf_] = values[np.argmax(counts)] # np.argmax(counts) finds the index of the highest value in counts (i.e., the most frequent class) => values[np.argmax(counts)] gets the class label that corresponds to the most frequent count.
        # target_class speichert also für jedes leaf, welches class label am meisten vorkommt


    # Prepare Feature Selection Output
    # Creates a dictionary var_a: Each key (1, 2, 3) corresponds to a feature selection vector. 1 at the position of the selected feature, 0 otherwise.
    # Each value is a list (length = number of features, len(P)), where: 1 marks the selected feature for that split; 0 marks all other features
    # var_a is a more structured and general way to represent the feature selection
    # var_a represents feature selection over all features in a way some models want it 

    var_a = {1: [0 if i != first_level else 1 for i in P], # Represents the feature selected for the root split
             2: [0 if i != left_second_level else 1 for i in P], # Represents the feature selected for the left branch of the second split
             3: [0 if i != right_second_level else 1 for i in P] # Represents the feature selected for the right branch of the second split
             }
    
    del df_arr


    logging.info(f'Training done. Loss: {m.objective.value()}\n'
                 f'Optimization status: {status}\n')
    



    # collects model statistics and results
    details = {
        'run_time': runtime,
        #'mip_gap': m.MIPGap,
        'objective': m.objective.value(),
        'status': status,
        'target_class': target_class,
        "var_a": var_a,
        "selected_features": {1: first_level,
                              2: left_second_level,
                              3: right_second_level
                              }
    }
    model_dict["details"] = details
    logging.info("Training is done.")
    return model_dict

In [ ]:
main_model_pulp = train_model_pulp(model_dict=main_model_pulp, data=train, P=P)



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/2223a1f43eb642edbfdbccb5aa7242f5-pulp.mps -sec 1800 -timeMode elapsed -branch -printingOptions all -solution /tmp/2223a1f43eb642edbfdbccb5aa7242f5-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 137 COLUMNS
At line 168144 RHS
At line 168277 BOUNDS
At line 202078 ENDATA
Problem MODEL has 132 rows, 33800 columns and 67600 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 1800
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.466895 - 0.01 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 0.466895 to -1.79769e+308
Probing was tried 0 times

In [ ]:
main_model_pulp_details = main_model_pulp["details"]

selected_features = main_model_pulp_details["selected_features"]
target_class = main_model_pulp_details["target_class"]

print("root feature is " + str(selected_features[1]))
print("left child feature is " + str(selected_features[2]))
print("right child feature is " + str(selected_features[3]))

print(target_class)

root feature is 112
left child feature is 50
right child feature is 111
{4: np.int64(2), 5: np.int64(3), 6: np.int64(3), 7: np.int64(2)}


# 4 Für neue Daten nach Modeltraining: Jede Entität (Zeile) durch model Lösung einem Blatt und damit Label zuordnen
dabei zeigt sich dann natürlich wie gut die Lösung ist

In [ ]:
#result = predict_model(data=train, model_dict=main_model, P=P)

#predicts the class for each instance in data as DataFrame using decision tree model, and marks at which leaf node each instance ends up in.
def predict_model_pulp(data: pd.DataFrame,
                  P: list,
                  model_dict: dict,
                  pruned_nodes: list = []) -> pd.DataFrame:
    """

    :param model_dict:
    :param pruned_nodes:
    :param data:
    :param P:
    :return:
    """

    prediction = []
    leaf_ = []
    depth = model_dict["depth"] # defined as depth = 2 in generate_model_pulp and so depth 2 was hardcoded there from the beginning
    var_a = model_dict["details"]["var_a"] #dict with all features for every of the 3 split decisions. Contains just a singular 1 at the index of the feature that was chosen
    target_class = model_dict["details"]["target_class"] #dict with 4 entries 1-4 (leafs of 2depth tree) as keys. The respective values are the labels of datapoints that were put there most (majority vote) 
    
    
    # Verteile rows auf die leafs
    for idx, i in data.iterrows(): #iterates over rows of dataframe
        x = np.array(i[P]) #array of all feature values \in {0,1} of that row
        #print(x)
        t = 1
        d = 0
        while d < depth: #traversing tree until getting the leaves
            at = np.array(var_a[t]) #binary vector over all features (P) with 1 where feature was chosen by solver
            #print(at)
            if at.dot(x) == 1: #dot product (sum of the element-wise); überprüft also ob x und at am gleichen Index eine 1 stehen haben. Das bedeutet also, dass wir in eine "ja" Instanz gehen, also in den linken Teilbaum
                t = t * 2
            else: # sonst gehen wir in den rechten Teilbaum
                t = t * 2 + 1
            d = d + 1
            if t in pruned_nodes:
                break
        prediction.append(target_class[t]) #prediction (target label determined by model solver) for this row
        leaf_.append(t) #respective leaf to prediction (jedes leaf steht ja für ein target label)
    data["prediction"] = prediction #attach to data dataframe
    data["leaf"] = leaf_ #attach to data dataframe
    logging.info("Prediction is done.")
    return data

In [ ]:
result_train = predict_model_pulp(data=train, model_dict=main_model_pulp, P=P)
print(result_train)

# das hier leafs missklassifiziert weden ergibt natürlich sinn, weil wir einen 2 level lookahead erzwingen und keinen optimalen decision tree auf den Daten
# => dieser 2 level lookahead tree wird dann building block für den rolling ansatz

     y  1  2  3  4  5  6  7  8  9  ...  123  124  125  126  127  128  129  \
0    1  0  0  0  0  0  1  0  0  0  ...    0    0    0    0    0    0    0   
1    1  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2    1  0  0  0  0  0  1  0  0  0  ...    0    0    0    0    0    1    0   
3    1  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
4    1  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
..  .. .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
155  3  0  0  0  0  1  0  0  0  0  ...    0    1    0    0    0    0    0   
156  3  0  0  0  0  0  0  0  1  0  ...    0    0    0    0    0    1    0   
157  3  0  0  0  0  0  0  1  0  0  ...    0    0    0    0    0    0    1   
158  3  0  0  0  0  0  1  0  0  0  ...    0    0    0    0    0    0    1   
159  3  0  0  0  0  0  0  0  0  0  ...    0    0    0    1    0    0    0   

     130  prediction  leaf  
0      0           2     7  
1      0         

# 5 Missclassification berechnen


In [ ]:
# misclassified_leafs = find_misclassification(df=result)

# identifies all leaf nodes where at least two different values of "y" (the target variable) are present in the training or prediction from model
# identifies which leaf nodes contain more than one unique value of the target variable "y", so impurity of the leaf
def find_misclassification(df: pd.DataFrame) -> list:
    """

    :param df:
    :return:
    """
    return list(df.loc[
                    df.groupby("leaf")["y"].transform(lambda x: (x.nunique() >
                                                                 1)),
                    'leaf'].unique())

In [ ]:
misclassified_leafs = find_misclassification(df=result_train)

print(misclassified_leafs)

[np.int64(7)]


In [ ]:
result_test = predict_model_pulp(data=test, model_dict=main_model_pulp, P=P)

print(result_test)

    y  1  2  3  4  5  6  7  8  9  ...  123  124  125  126  127  128  129  130  \
0   1  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   
1   1  0  0  0  0  0  1  0  0  0  ...    0    0    0    0    0    0    0    0   
2   1  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0    0   
3   1  0  0  0  0  0  0  0  0  1  ...    0    0    0    0    0    0    0    0   
4   1  0  0  0  0  0  0  0  1  0  ...    0    0    0    0    0    0    1    0   
5   1  0  0  0  0  0  0  0  0  1  ...    0    0    0    0    0    0    0    0   
6   2  0  0  0  0  1  0  0  0  0  ...    1    0    0    0    0    0    0    0   
7   2  0  0  0  1  0  0  0  0  0  ...    0    1    0    0    0    0    0    0   
8   2  0  1  0  0  0  0  0  0  0  ...    0    0    1    0    0    0    0    0   
9   2  0  0  0  1  0  0  0  0  0  ...    0    0    1    0    0    0    0    0   
10  2  0  1  0  0  0  0  0  0  0  ...    0    0    0    1    0    0    0    0   
11  2  1  0  0  0  0  0  0  

In [ ]:
train_acc = len(result_train.loc[result_train["prediction"] == result_train["y"]]) / len(result_train["y"])


test_acc = len(result_test.loc[result_test["prediction"] == result_test["y"]]) / len(result_test["y"])

#del result_test

result_dict = {}
result_dict['tree'] = {}
result_dict['tree'][2] = {}
result_dict['tree'][2]['train'] = result_train[['y', 'prediction', 'leaf']]
result_dict['tree'][2]['test'] = result_test[['y', 'prediction', 'leaf']]

In [ ]:
print(result_dict)

{'tree': {2: {'train':      y  prediction  leaf
0    1           2     7
1    1           2     7
2    1           2     7
3    1           2     7
4    1           2     7
..  ..         ...   ...
155  3           3     5
156  3           3     5
157  3           3     6
158  3           3     6
159  3           3     5

[160 rows x 3 columns], 'test':     y  prediction  leaf
0   1           2     7
1   1           2     7
2   1           2     7
3   1           2     7
4   1           2     7
5   1           2     7
6   2           2     7
7   2           2     7
8   2           2     7
9   2           2     7
10  2           2     7
11  2           2     7
12  2           2     7
13  3           3     6
14  3           3     5
15  3           2     7
16  3           3     5
17  3           3     5}}}


In [ ]:
train = train.drop(["prediction", "leaf"], axis=1) # remove selected coloumns (axis=1 for coloumns)
test = test.drop(["prediction", "leaf"], axis=1)

rolling_optimize also die konkrete Implementierung für den rolling algorithmus habe ich noch nicht verstanden!

hier muss man noch was tun, um zu sehen was am Ende in den Blättern landet.
**Nachschauen was die random forst api (sklearn wrapper) braucht**

![alt text](image.png)

watch out for slight differences in naming using _ e.g.: to_go_deep_nodes is different to to_go_deep_nodes_

In [ ]:
from typing import Optional
#import time

#from rollo_oct.oct.optimal_tree import predict_model, generate_model, train_model
from tree import *


def _rolling_optimize(predefined_model: Optional[dict],
                      train_data: pd.DataFrame,
                      test_data: pd.DataFrame,
                      main_depth: int,
                      target_depth: int,
                      features: list,
                      time_limit: float,
                      to_go_deep_nodes: list,
                      result_dict: dict,
                      criterion: str = "gini"
                      ) -> pd.DataFrame:
    """

    :param predefined_model:
    :param train_data:
    :param test_data:
    :param main_depth:
    :param target_depth:
    :param features:
    :param time_limit:
    :return:
    """

    model_name = f"rolling_optimizing_gini_{main_depth}"  #why only gini?
    logging.info(f"Running {model_name} with main depth {main_depth} by "
                 f"diving in "
                 f"2 level with target depth {target_depth}")

    df_arr = np.array(train_data)
    leaf_nodes_path = predefined_model["nodes"]["leaf_nodes_path"] # {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}
    y_idx = 0
    predefined_model.pop('model')
    predefined_model.pop('params')
    final_model = copy.deepcopy(predefined_model)


    selected_features_temp = copy.deepcopy(predefined_model["details"][ # z.B. {1: 112, 2: 50, 3: 111}
                                               "selected_features"])
    selected_features = {1: selected_features_temp} # z.B. {1: {1: 112, 2: 50, 3: 111}}
    to_go_deep_nodes = {i: i for i in to_go_deep_nodes} # keeps deep nodes (parents of impure leafs). z.B {np.int64(7): np.int64(7)}
    pruned_nodes = [i for i in leaf_nodes_path if i not in to_go_deep_nodes] #all nodes not in to_go_deep_nodes z.b. [4, 5, 6]
    
    
    new_train_data_dict = {}

    
    for level in range(1, target_depth - main_depth + 1): #for each level of the rolling tree, "breadth first search approach"
        logging.info(f"Extending tree from Depth {main_depth} to "
                     f"{main_depth + level}")
        iter_time = time.time()
        to_go_deep_nodes_ = {}
        sub_K = {}

        current_depth = main_depth + level
        #print(current_depth)

        parents_to_optimize = parents_of_nodes_to_branch_on( #gives parent of impure node; e.g. {3: [np.int64(7)]} because node 3 ist parent of node 7 (leaf) that is impure
            go_deep_nodes=to_go_deep_nodes)
        

        for node_ in parents_to_optimize:
            leafs_ = parents_to_optimize[node_] # gets the [np.int64(7)] from the example {3: [np.int64(7)]}
            sub_features = selected_features[int(node_ / 2)]

            if level == 1:
                first_var = leaf_nodes_path[leafs_[0]][0]
                arr = df_arr[np.where((df_arr[:, sub_features[1]] ==
                                       first_var))]

                sub_K[node_] = list(np.unique(arr[:, y_idx]))
                cols = features.copy()
                cols.insert(y_idx, 'y')
                new_train_data_dict[node_] = pd.DataFrame(
                    arr, columns=cols,
                    index=None)
            else:
                arr = new_train_data_dict[int(node_ / 2)]
                arr = np.array(arr)
                first_var = leaf_nodes_path[to_go_deep_nodes[leafs_[0]]][0]
                arr = arr[np.where((arr[:, sub_features[1]] ==
                                    first_var))]
                sub_K[node_] = list(np.unique(arr[:, y_idx]))
                cols = features.copy()
                cols.insert(y_idx, 'y')
                new_train_data_dict[node_] = pd.DataFrame(
                    arr, columns=cols,
                    index=None)

            logging.info(
                f"Processing for Parent Node: {node_} of Leaf "
                f"Node(s): {leafs_} at Level: {level}")
            main_model = generate_model_pulp(P=features,
                                        K=sub_K[node_],
                                        data=new_train_data_dict[node_],
                                        y_idx=y_idx,
                                        time_limit=time_limit,
                                        log_to_console=False,
                                        criterion=criterion)
            main_model = train_model_pulp(model_dict=main_model,
                                     data=new_train_data_dict[node_],
                                     P=features)
            result_ = predict_model_pulp(
                data=new_train_data_dict[node_],
                model_dict=main_model,
                P=features)
            misclassified_leafs = find_misclassification(df=result_)
            del result_
            temp_dict = {}
            for t in predefined_model["nodes"]["parent_nodes"]:
                new_idx = parent_pattern(sub_leaf=t,
                                         leaf_node=node_)
                temp_dict[new_idx] = main_model[
                    "details"]["var_a"][t]
            main_model["details"]["var_a"] = temp_dict


            # update extended tree's target class
            temp_target = {}
            for t in predefined_model["nodes"]["leaf_nodes"]:
                new_idx = leaf_pattern(sub_leaf=t, depth=main_depth,
                                       leaf=node_)
                if t in main_model["details"]["target_class"]:
                    # less than 3 data points results with error
                    temp_target[new_idx] = \
                        main_model["details"]["target_class"][t]
                    if t in misclassified_leafs:
                        to_go_deep_nodes_[new_idx] = t
                    else:
                        pruned_nodes.append(new_idx)



            # reorganize pruned nodes - if one of the leaf has
            # misclassified, then its parent will reoptimize.
            temp_class_assign = copy.deepcopy(pruned_nodes)
            for i in temp_class_assign:
                for parent in parents_to_optimize:
                    if not (i != parent * 2 and i != (
                            parent * 2 +
                            1)):
                        pruned_nodes.remove(i)
                    if not (i != get_child(1, 2, parent) * 2 or i != (
                            get_child(1, 2, parent) * 2 +
                            1)):
                        pruned_nodes.remove(i)
            new_train_data_dict[node_] = new_train_data_dict[node_].drop(
                ["prediction",
                 "leaf"],
                axis=1)
            


            # eliminate branched on node from target class
            main_model["details"]["target_class"] = temp_target
            selected_features[node_] = copy.deepcopy(
                main_model["details"][
                    "selected_features"])
            final_model["details"]["var_a"].update(
                main_model["details"]["var_a"])
            for k in [node_ * 2, node_ * 2 + 1]:
                if k in final_model["details"]["target_class"]:
                    final_model["details"]["target_class"].pop(k)
            final_model["details"]["target_class"].update(
                main_model["details"]["target_class"])

        ####### hier ist drin wie die train daten im aktuellen Teilbaum performen; dict in predict_model_pulp)
        
        final_model["depth"] = main_depth + level
        result_training_data = predict_model_pulp(data=train_data,
                                model_dict=final_model,
                                P=features,
                                pruned_nodes=pruned_nodes)
        #######################

        result_dict['tree'][current_depth] = {}
        
        result_dict['tree'][current_depth]['train'] = result_training_data[['y', 'prediction', 'leaf']] #adding dict to save classification for every level

        train_acc = len(
            result_training_data.loc[result_training_data["prediction"]
                        == result_training_data["y"]]) / \
                    len(result_training_data["y"])
        #del result_training_data

        ####### hier ist drin wie die test daten im aktuellen Teilbaum performen; dict in predict_model_pulp)
        result_test_data = predict_model_pulp(data=test_data,
                               model_dict=final_model,
                               P=features,
                               pruned_nodes=pruned_nodes)
        
        ################################################

        result_dict['tree'][current_depth]['test'] = result_test_data[['y', 'prediction', 'leaf']]
        
        prediction_acc = len(
            result_test_data.loc[result_test_data["prediction"]
                       == result_test_data["y"]]) / \
                         len(result_test_data["y"])
        logging.info(
            f"Test Accuracy: {prediction_acc}. Training Accuracy: "
            f"{train_acc}. Iteration is over for level"
            f" {level}. Final "
            f"depth is {main_depth + level}")
        to_go_deep_nodes = to_go_deep_nodes_

        
        result_dict[current_depth] = {
            "training_accuracy": train_acc,
            "test_accuracy": prediction_acc,
            "time": time.time() - iter_time

        }
    #return result_dict, result_test_data[['y', 'prediction', 'leaf']], result_training_data[['y', 'prediction', 'leaf']]
    return result_dict


In [ ]:
#print(depth)
if depth > 2:
        result_dict = _rolling_optimize(predefined_model=main_model_pulp,
                                        train_data=train,
                                        test_data=test,
                                        main_depth=2,
                                        target_depth=depth,
                                        features=P,
                                        time_limit=time_limit,
                                        to_go_deep_nodes=misclassified_leafs,
                                        result_dict = result_dict,
                                        criterion=criterion)
          # add main model


{}
{}
{'leaf_nodes': [4, 5, 6, 7], 'leaf_nodes_path': {4: [1, 1], 5: [1, 0], 6: [0, 1], 7: [0, 0]}}
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/drood/Obsidian/Files/Bachelorarbeit/rlrf_my_try/.venv/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/137da39389b64a1da359534c035d65ef-pulp.mps -sec 1800 -timeMode elapsed -branch -printingOptions all -solution /tmp/137da39389b64a1da359534c035d65ef-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 137 COLUMNS
At line 168129 RHS
At line 168262 BOUNDS
At line 202063 ENDATA
Problem MODEL has 132 rows, 33800 columns and 67600 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 1800
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0.419048 - 0.01 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothin

KeyError: 'tree'

In [ ]:
# so wie im code zwischen rolling tree depth <= 2 und größer als 2 unterschieden wird, muss man verschieden die results dann zurückgeben. Für tiefe 2 muss man die results aus der "vorbarbeit" zurückgeben

result_dict[2] = {
    "training_accuracy": train_acc,
    "test_accuracy": test_acc,
    #"time": time.time() - main_model_time
}

print(result_dict)

{3: {'training_accuracy': 0.69375, 'test_accuracy': 0.7777777777777778, 'time': 2.473400592803955}, 4: {'training_accuracy': 0.75, 'test_accuracy': 0.8333333333333334, 'time': 2.247697114944458}, 5: {'training_accuracy': 0.8375, 'test_accuracy': 0.8888888888888888, 'time': 3.6779839992523193}, 6: {'training_accuracy': 0.9, 'test_accuracy': 0.9444444444444444, 'time': 2.2073781490325928}, 7: {'training_accuracy': 0.94375, 'test_accuracy': 0.9444444444444444, 'time': 2.17445707321167}, 8: {'training_accuracy': 0.96875, 'test_accuracy': 0.9444444444444444, 'time': 3.6705567836761475}, 2: {'training_accuracy': 0.5875, 'test_accuracy': 0.6111111111111112}}


number of nodes in perfect binary tree of dpeth k: $N = 2^{k+1} - 1$ 

numbering of leaves ist consistent with numbering in complete binary tree starting with 1 at the root and counting nodes level by level

depth $k$: max node number $N = 2^{k+1} - 1$; leaf nodes from  $N+1 - 2^k$ to N , alternative $N+1 = 2^{k+1}$

depth $k=2$: max node number $N = 2^{2+1} - 1 = 7$; leaf nodes from  $N+1 - 2^k$ to N => $7+1 - 2^2 = 8-4 = 4$ to 7

depth $k=3$: max node number $N = 2^{3+1} - 1 = 15$; leaf nodes from  $N+1 - 2^k$ to N => $15+1 - 2^3 = 16-8 = 8$ to 15

depth $k=4$: max node number $N = 2^{4+1} - 1 = 31$; leaf nodes from  $N+1 - 2^k$ to N => $31+1 - 2^4 = 32-16 = 16$ to 31

...

In [ ]:
if depth > 2:
    print(result_df_training_data)
else: 
    print(result_train[['y', 'prediction', 'leaf']])

     y  prediction  leaf
0    1           1   125
1    1           1    61
2    1           2   511
3    1           1    13
4    1           1    13
..  ..         ...   ...
155  3           3     5
156  3           3     5
157  3           3   510
158  3           3   510
159  3           3     5

[160 rows x 3 columns]


In [ ]:
if depth > 2:
    print(result_df_test_data)
else: 
    print(result_test[['y', 'prediction', 'leaf']])

    y  prediction  leaf
0   1           1    13
1   1           1    13
2   1           1    61
3   1           1   125
4   1           2   511
5   1           1    13
6   2           2    12
7   2           2   511
8   2           2   509
9   2           2    58
10  2           2   511
11  2           2   511
12  2           2   511
13  3           3   510
14  3           3     5
15  3           3    59
16  3           3     5
17  3           3     5
